# Notebook 27 - Generate Deployment Instructions and KPIs

This notebook produces final deployment-ready summaries per store to assist real-world implementation and final reporting.

It consolidates:
- Top-matching recipes per store (based on waste-aware priority score)
- Waste impact potential per recipe
- Full inventory of matched product articles
- Summary KPIs per store for presentation and deployment

These outputs are designed for use by operational staff, retail decision-makers, and inclusion in the thesis results chapter.

## Objectives
- Load final ranked meal box plan and inventory matches
- Aggregate deployment summaries per store
- Export a deployment pack including per-store CSVs and a summary table
- Optional: Render LaTeX-ready and presentation-ready KPIs

## Inputs
- `recipe_ranking/store_mealbox_ranked.csv` - Ranked store–recipe matches
- `store_inventory_exports/*.csv` - Product-level inventory for each store
- `waste_simulation/waste_impact_simulated.csv` - Waste reduction estimates

## Outputs
- `deployment_exports/store_deployment_summary.csv` - Main KPI overview
- `deployment_exports/per_store/` - Folder of per-store CSV instruction files


In [4]:
# Code
import os
import pandas as pd

# Define folders
ranking_folder = "recipe_ranking"
inventory_folder = "store_inventory_exports"
impact_folder = "waste_simulation"
output_folder = "deployment_exports"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(os.path.join(output_folder, "per_store"), exist_ok=True)

# Define key files
ranked_file = os.path.join(ranking_folder, "recipe_store_ranked.csv")
impact_file = os.path.join(impact_folder, "waste_impact_simulated.csv")


In [5]:
# Code
# Load ranked recipe-store pairs
df_ranked = pd.read_csv(ranked_file)
df_impact = pd.read_csv(impact_file)

print("Loaded:")
print(f"- Ranked store-recipe pairs: {df_ranked.shape}")
print(f"- Waste impact data: {df_impact.shape}")


Loaded:
- Ranked store-recipe pairs: (4, 6)
- Waste impact data: (4, 3)


In [6]:
# Merge in waste impact data (per store)
df_ranked = df_ranked.merge(df_impact, on="store", how="left")

# Confirm merge
assert "value_wasted" in df_ranked.columns
print("Merged waste impact into ranked mealbox plan.")


Merged waste impact into ranked mealbox plan.


In [7]:
# Load all per-store CSVs into a dictionary
inventory_files = [
    f for f in os.listdir(inventory_folder) if f.endswith(".csv")
]

store_inventories = {}

for file in inventory_files:
    store_id = os.path.splitext(file)[0].split("_")[-1]
    store_id = int(store_id)
    df_inv = pd.read_csv(os.path.join(inventory_folder, file))
    store_inventories[store_id] = df_inv

print(f"Loaded inventory files for {len(store_inventories)} stores.")


Loaded inventory files for 9 stores.


In [9]:
# Build summary per store (with robust column normalization)
summary_records = []

for store_id, df_inv in store_inventories.items():
    # Normalize column names to lowercase
    df_inv.columns = df_inv.columns.str.strip().str.lower()

    # Safely access columns
    recipes = df_inv["recipe"].nunique() if "recipe" in df_inv.columns else 0
    ingredients = df_inv["ingredient"].nunique() if "ingredient" in df_inv.columns else 0
    products = df_inv["product_article"].nunique() if "product_article" in df_inv.columns else 0

    # Get value saved
    waste_row = df_ranked[df_ranked["store"] == store_id]
    waste_value = float(waste_row["value_wasted"].values[0]) if not waste_row.empty else 0

    summary_records.append({
        "store": store_id,
        "recipes_planned": recipes,
        "unique_ingredients": ingredients,
        "products_to_pick": products,
        "estimated_value_saved": round(waste_value, 2)
    })

df_summary = pd.DataFrame(summary_records)
df_summary = df_summary.sort_values("estimated_value_saved", ascending=False)

print("Generated store deployment summary:")
display(df_summary)


Generated store deployment summary:


,store,recipes_planned,unique_ingredients,products_to_pick,estimated_value_saved
0,1024,1,0,0,0.79
1,1058,1,0,0,0.00
2,1160,1,0,0,0.00
3,3298,1,0,0,0.00
4,3345,1,0,0,0.00
5,4243,1,0,0,0.00
6,4260,1,0,0,0.00
7,4278,1,0,0,0.00
8,5070,1,0,0,0.00


In [10]:
# Save the KPI summary for use in the thesis and reporting
summary_file = os.path.join(output_folder, "store_deployment_summary.csv")
df_summary.to_csv(summary_file, index=False)

print("Saved store deployment summary to:", summary_file)


Saved store deployment summary to: deployment_exports\store_deployment_summary.csv


In [11]:
# Export per-store deployment files
for store_id, df_inv in store_inventories.items():
    df_inv.columns = df_inv.columns.str.strip().str.lower()
    output_path = os.path.join(output_folder, "per_store", f"deployment_store_{store_id}.csv")
    df_inv.to_csv(output_path, index=False)

print(f"Exported {len(store_inventories)} per-store deployment files.")


Exported 9 per-store deployment files.
